# Install required Python modules

*Note:* This step may take a while to run the first time.

In [1]:
# Step 1: Install necessary libraries (run in a cell if needed)
!pip install -q -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from pymilvus import connections, utility, Collection, CollectionSchema, FieldSchema, DataType
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

/opt/app-root/lib64/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# Shakeout minio connection

In [3]:
#!/usr/bin/env python3
"""
Shake-out test for a MinIO deployment on Kubernetes.

Environment variables:
  AWS_S3_ENDPOINT        – MinIO service DNS name (e.g. minio.minio.svc.cluster.local)
  AWS_ACCESS_KEY_ID      – MinIO access key
  AWS_SECRET_ACCESS_KEY  – MinIO secret key
  AWS_DEFAULT_REGION     – Dummy value; boto3 still expects one
"""
import os
import sys

import boto3
from botocore.client import Config
from botocore.exceptions import BotoCoreError, ClientError


endpoint = os.getenv("AWS_S3_ENDPOINT", "minio.minio.svc.cluster.local")
access_key = os.getenv("AWS_ACCESS_KEY_ID", "minio")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY", "test")
region = os.getenv("AWS_DEFAULT_REGION", "us-east-1") or "us-east-1"


minio_status = "🟢 OK"

try:
    s3 = boto3.client(
        "s3",
        endpoint_url=f"http://{endpoint}",
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_key,
        region_name=region,
        config=Config(signature_version="s3v4"),
    )

    resp = s3.list_buckets()
    buckets = [b["Name"] for b in resp.get("Buckets", [])]

    if buckets:
        print("🟢 Connection succeeded – buckets discovered:")
        for name in buckets:
            print(f"  • {name}")
    else:
        print("🟢 Connected but no buckets found.")

except (BotoCoreError, ClientError) as exc:
    print(f"🔴 MinIO connectivity test failed: {exc}", file=sys.stderr)
    minio_status="🔴 FAIL"

🟢 Connection succeeded – buckets discovered:
  • data
  • models
  • rag-docs


# Shakeout the Milvus connectivity


## Create the Milvus database

In [4]:
milvus_status = "🟢 OK"
# This is the name of the collection that this program will use.
collection_name = "shakeout_collection"

try:
    # Create the client object
    connections.connect(
        uri="http://milvus-service.milvus.svc.cluster.local:19530",
        alias="default"
    )
    
    # Make sure we start with a clean slate by deleting the collection if it exists from a prior run.
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    print(f"Collection list: {utility.list_collections()}") 

    # Databases need a schema. In this lab the schema will consist 
    # of an identifier and a vector that contains the embedding of a text string.
    
    # Define the primary key field for unique record identification
    id_field = FieldSchema(
        name="id",
        dtype=DataType.INT64,
        is_primary=True,
        auto_id=False
    )
    
    # Specify embedding model and its output dimension
    embedding_model = "all-MiniLM-L6-v2"  # Example Hugging Face model
    embedding_dim = 384  # Embedding vector size as per the model
    
    # Define the vector field to hold embedding values
    embedding_field = FieldSchema(
        name="embedding",
        dtype=DataType.FLOAT_VECTOR,
        dim=embedding_dim
    )
    
    # Assemble collection schema combining ID and embedding fields
    schema = CollectionSchema(
        fields=[id_field, embedding_field],
        description="Milvus shakeout test",
        enable_dynamic_field=False
    )

    # Instantiate the Milvus collection using the defined schema and configuration
    collection = Collection(
        name=collection_name, 
        schema=schema, 
        using='default', 
        shards_num=2,
        consistency_level="Strong"
    )
    
    # List all collections in Milvus to confirm creation
    print(f"Collection list: {utility.list_collections()}")

    # Close the Milvus connection
    collection.release()
    utility.drop_collection(collection_name)
except:
    milvus_status="🔴 FAIL"

Collection list: ['animal_test', 'my_rag_collection']
Collection list: ['animal_test', 'my_rag_collection', 'shakeout_collection']


In [5]:
print(f"Minio status: {minio_status}")
print(f"Milvus status: {milvus_status}")

Minio status: 🟢 OK
Milvus status: 🟢 OK
